<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_5_rapids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

**Module 4: Training for Tabular Data**

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).


# Module 4 Material

- Part 4.1: Using K-Fold Cross-validation with PyTorch [[Video]](https://www.youtube.com/watch?v=Q8ZQNvZwsNE&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_1_kfold.ipynb)
- Part 4.2: Training Schedules for PyTorch  [[Video]](https://www.youtube.com/watch?v=lMMlbmfvKDQ&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_2_schedule.ipynb)
- Part 4.3: Dropout Regularization [[Video]](https://www.youtube.com/watch?v=4ixjgw6Q42U&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_3_dropout.ipynb)
- Part 4.4: Batch Normalization [[Video]](https://www.youtube.com/watch?v=1U5nOKh9OLQ&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_4_batch_norm.ipynb)
- **Part 4.5: RAPIDS for Tabular Data** [[Video]](https://www.youtube.com/watch?v=KgoXuhG_kfs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_04_5_rapids.ipynb)


# Installing Rapids on Google Colab

In this section, we're going to explore the use of NVIDIA RAPIDS, a suite of software libraries and APIs which enables us to execute end-to-end data science and analytics pipelines entirely on GPUs. This powerful tool leverages the compute capabilities of GPUs to provide significant performance improvements over traditional CPU-based workflows. However, it's important to note that to use RAPIDS effectively, a compatible GPU is necessary. As we progress, we suggest that you run the example provided in Google Colab. Google Colab is a free cloud service that supports GPU-based computation, and our example has been crafted to run optimally in this environment.

If you wish to utilize RAPIDS in a different environment, be aware that this requires some specialized installation steps. This might involve installing appropriate GPU drivers, configuring CUDA (a parallel computing platform and API by NVIDIA), and installing RAPIDS libraries. While it is entirely possible to set up RAPIDS in such environments, it does require a certain level of familiarity with system administration and is beyond the scope of this book. We encourage you to follow the latest installation guides provided on the official RAPIDS website if you choose this route.

The following code installs RAPIDS into Google Colab.


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

# Part 4.5: RAPIDS for Tabular Data

In this chapter where we delve into the powerful world of NVIDIA RAPIDS for machine learning with tabular data. As we increasingly engage with large volumes of data in modern machine learning, efficient processing and manipulation become critical. This is where RAPIDS steps in. This suite of open-source software libraries, developed by NVIDIA, accelerates data science pipelines by leveraging the power of GPUs. In this chapter, we'll explore how RAPIDS can be used to transform, model, and infer from tabular data in a significantly reduced timeframe compared to traditional CPU-based methods. Whether you're predicting sales, analyzing customer behavior, or detecting anomalies, using RAPIDS with tabular data can dramatically streamline your workflow and enhance your machine learning models' performance.

## Deep Learning and Tabular Data: Not Always a Perfect Match

While deep learning has brought transformative changes in numerous fields, such as image recognition, natural language processing, and even playing board games, it has sometimes fallen short when applied to tabular data, especially in comparison to traditional machine learning algorithms.

It's important to understand that neural networks, the core of deep learning, are entirely capable of performing regression and classification tasks on tabular data. By applying various network architectures and tuning hyperparameters, you can certainly train a deep learning model on such datasets.

However, in many instances, more traditional models, such as XGBoost, Random Forests, and other Gradient Boosted Machines (GBMs) often outperform deep learning models on purely tabular datasets. These traditional models have shown their efficiency in handling tabular data due to their ability to better capture certain kinds of relationships and dependencies between variables that neural networks sometimes miss.

Real-world evidence of this trend can be found in the results of various Kaggle competitions. Kaggle, a platform for predictive modelling and analytics competitions, offers a wealth of data on machine learning model performance. Many winners of these competitions, particularly those focused on tabular data, frequently utilize GBMs. For instance, the XGBoost algorithm, a scalable and accurate implementation of gradient boosting machines, has been a part of numerous winning solutions.

One reason for this is that tabular data often has structured and hierarchical relationships, and gradient boosting algorithms excel at capturing these interactions. On the other hand, deep learning models, particularly those without specific architecture designs, might struggle with such data unless provided with large amounts of training data and carefully selected features.

This is not to say that deep learning should be disregarded when dealing with tabular data. Rather, it's important to understand the strengths and weaknesses of each approach. In some cases, combining deep learning with traditional methods can result in very powerful models. However, for most tabular datasets, the traditional models have proven to be more reliable, accurate, and computationally efficient.

It's essential to remember that in machine learning, there is no "one size fits all" solution. The effectiveness of a model depends on the nature of the problem, the quality and quantity of the data, and the computational resources available. As practitioners, our role is to navigate this landscape and select or design the best models for our specific needs.

## NVIDIA RAPIDS and Random Forests for Tabular Prediction

As we a completly GPU-based pipeline, we're going to adapt a previous PyTorch neural network example to make use of NVIDIA RAPIDS. While neural networks have their strengths, there are cases where traditional models like Random Forests can outperform them, especially when dealing with certain types of tabular data. In our case, we've found that for the dataset we're working with, a Random Forest model indeed surpasses the previously used neural network in performance.

In this section, we'll guide you step-by-step on how to replace our neural network with a Random Forest, demonstrating the flexibility and versatility that RAPIDS offers when it comes to choosing and changing machine learning models.

A highlight of this adaptation will be the use of RAPIDS' cudf library. This is a powerful GPU-accelerated data manipulation library that allows us to perform all necessary preprocessing of our dataset entirely within GPU memory. One of the significant advantages here is that we can directly hand off our processed data to the Random Forest model without leaving GPU memory, enhancing the efficiency of our data science pipeline.

We'll demonstrate how to take full advantage of these RAPIDS features, optimizing our model's performance and speed. By the end of this section, you'll not only have a stronger understanding of how to adapt different machine learning models within RAPIDS but also have gained hands-on experience in leveraging GPU memory for efficient data processing and modeling. This code is demonstrated here.


In [ ]:
import cudf as pd
from cuml import train_test_split
from cuml.ensemble import RandomForestRegressor
from cuml.metrics import mean_squared_error
from cuml.preprocessing import StandardScaler


# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=["NA", "?"],
)

# Generate dummies for job
df = pd.concat([df, pd.get_dummies(df["job"], prefix="job", dtype=int)], axis=1)
df.drop("job", axis=1, inplace=True)

# Generate dummies for area
df = pd.concat([df, pd.get_dummies(df["area"], prefix="area", dtype=int)], axis=1)
df.drop("area", axis=1, inplace=True)

# Generate dummies for product
df = pd.concat([df, pd.get_dummies(df["product"], prefix="product", dtype=int)], axis=1)
df.drop("product", axis=1, inplace=True)

# Missing values for income
med = df["income"].median()
df["income"] = df["income"].fillna(med)

# Standardize ranges
scaler = StandardScaler()
df["income"] = scaler.fit_transform(df["income"].to_frame())
df["aspect"] = scaler.fit_transform(df["aspect"].to_frame())
df["save_rate"] = scaler.fit_transform(df["save_rate"].to_frame())
df["subscriptions"] = scaler.fit_transform(df["subscriptions"].to_frame())

# Convert to cuDF DataFrame
x_columns = df.columns.drop(["age", "id"])
x = df[x_columns]
y = df["age"]
x = x.astype("float32")
y = y.astype("float32")

# Set random seed for reproducibility
import numpy as np

np.random.seed(42)

# Split into training and test datasets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# Train the model with RandomForestRegressor
model = RandomForestRegressor(n_estimators=500, random_state=42, n_streams=1)
model.fit(x_train, y_train)

# Test the model
y_pred = model.predict(x_test)

# Compute RMSE
score = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Final score (RMSE): {score}")